# Getting metrics for the first model

Let's try to update my report with the metrics for the first model.

As per the tests conducted on the first notebook, regarding the datasets (to find which datasets can be run in a reasonable time), we are going to use these default datasets:

* `birds`
* `emotions`
* `scene`

The **baseline** will be the regular Binary Relevance. Then we will compare those to the Basic Stacking approach. Finally, we will run the Stacking With F-Test, with `alpha=0.5`. All other parameters, of the other models, will be the default ones, using the `SVC` as the base classifier.

## Setup

In [ ]:
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.dataset import load_dataset, available_data_sets
from sklearn.svm import SVC
from skmultilearn.base.problem_transformation import ProblemTransformationBase
from typing import List, Optional, Any, Tuple, Dict
import numpy as np
import sklearn.metrics as metrics
import json
import pandas as pd
from sklearn.feature_selection import f_classif
